# <font color=blue>Контрактное програмирование</font>

Поиск ошибок в программе трудное и дорогостоящее занятие, поэтому есть методики, позволяющие снизить количество и облегчить написание программы. Одна из них это **контрактное программирование**.

Этот метод основан на четком определении взаимных обязательств элементов программы.

**Взаимодействующие элементы программы:**

- "клиент" - вызывающая функция, объект или модуль;

- "поставщик" - вызываемая функция, объект или модуль.

**"Контракт"** между ними - это взаимные обязательства и преимущества:

- обязательства - то, что каждой стороне требуется соблюсти при взаимодействии;

- преимущества - та выгода, которая приобретается при соблюдении обязательств другой стороной.

Архитектор программы определяет **формальные** **точные** и **верифицируемые** спецификации интерфейсов.

**Содержание контракта**

- предусловия - обязательства клиента перед вызовом функции,

- постусловия - обязательства функции-поставщика, которые должны быть выполнены при окончании ее работы.

- инварианты - условия, которые должны выполняться, как при вызове функции поставщика, так и при окончании его работы.

## <font color=green>Утверждения (Assertions)</font>

Синтаксис

```python
assert conditions, error_message
```

Условия `assert` проверяются если `__debug__ == True`.  Переменная `__debug__` выставляется при запуске интерпретатора и по умолчанию равна `True`. Её нельзя изменить в процессе выполнения программы. Чтобы отключить assertions используется опция `-O`. Сравните
```bash
python -O -c "print(__debug__); assert False"
python -c "print(__debug__); assert False"
```

Если утверждение не выполняется, то выбрасывается исключение `AssertionError`. **Никогда не перехватывайте эти исключения. Они указывают на ошибки в программе и не применяются для других целей.**

### Пример:

In [ ]:
def f(hour):
    assert 0 <= hour <= 23, "Hours should be in range 1..23"

In [ ]:
def f(hour):
    assert 0 <= hour <= 23, "Hours should be in range 1..23"

In [ ]:
print(__debug__)
f(-1)

In [ ]:
import sys
sys.modules[__name__].__debug__ = False
print(__debug__)
f(25)

Утверждения используются для проверки значений параметров функции и проверки возвращаемого значения.

In [ ]:
def fibo(n):
    assert n > 0, "index of Fibonacci number can not be less than 1"
    assert isinstance(n, int), "index of Fibonacci number has to be integer"
    a, b = 1, 1
    for _ in range(n-2):
        a, b = b, a + b
    return b

Для исключения разных невероятных ситуаций.

In [ ]:
def count_sort(L, rad):
    n = len(L)
    buf = [0]*n
    num_buskets = 11
    count = [0]*num_buskets
    exp = 10**rad
    for e in L:
        q = e // exp
        rem = q % 10
        if q == 0:
            count[0] += 1
        else:
            count[rem + 1] += 1
    for i in range(1, num_buskets):
        count[i] += count[i-1]
    # assertion
    assert count[-1] == len(L), \
        "sum of counts for all digits has to be equal" \
        " to length of the list being sorted"
    for i in range(n-1, -1, -1):
        e = L[i]
        q = e // exp
        rem = q % 10
        if q == 0:
            count[0] -= 1
            buf[count[0]] = e
        else:
            count[rem+1] -= 1
            buf[count[rem+1]] = e
    L[:] = buf

Для проверки пред- и постусловий лучше подходит библиотека `PyContracts`, поэтому `assert` используют для проверки инвариантов.

**Внутренний инвариант** (internal invariant) — это логическое выражение, выражающее уверенность программиста в значении некоторых переменных в некоторый момент выполнения программы.

In [ ]:
# пусть есть две возможности: остаток от деления `x` на 3 равен 0
# и остато от деления `x` на 3 равен 2. Чтобы убедиться в этом 
# удобно использовать `assert`

if x%3 == 0:
    print("Число делится на три.")
else:
    assert x%3 == 2  # assert здесь является комментарием, гарантирующим истинность утверждения
    print("Остаток при делении на три - два.")

**Инвариант потока выполнения** (control-flow invariants) — выражает уверенность программиста в том как идёт поток выполнения. В том числе, что какой-то участок кода никогда не должен быть достигнут.

In [ ]:
def get_fisrt_negative_in_list(L):
    for a in L:
        if a < 0:
            return a
    assert False

**Инвариант класса** (class invariant) — это семантические свойства и ограничения целостности экземпляра класса. Например, объект календарной даты никогда не может находиться в состоянии 31 апреля или 30 февраля. Объект класса красно-чёрного дерева поиска в момент вызова любого его метода, как и по окончании, должен быть сбалансирован.

##  <font color=green>Аннотация типов</font>

Начиная с Python 3.6 в язык была добавлена возможность указания типов параметров функции и типа ее возвращаемого значения.

In [ ]:
def fibo(n: int) -> int:
    a: int = 1
    b: int = 1
    if n < 3:
        return 1
    for _ in range(n-2):
        a, b = b, a+b
    return b

Аннотации типов ни к чему не обязывают. Если не будут выполнены специальные настройки, интерпретатор не выдаст ошибок или предупреждений. Тем не менее, аннотации полезны: они улучшают читаемость кода, а в некоторых IDE, например PyCharm, выдают предупреждения, если тип переменной не совпадает с аннотацией.

Для аннотации сложных объектов используется модуль `typing`. В `typing` есть объекты `List`, `Tuple`, `Dict`, `Str`, применяемые для аннотации соответствующих встроенных типов.

In [ ]:
from typing import List, Tuple, Dict

names: List[str] = ["Guido", "Jukka", "Ivan"]
version: Tuple[int, int, int] = (3, 7, 1)
options: Dict[str, bool] = {"centered": False, "capitalize": True}

## <font color=green>Библиотека `PyContracts`</font>

Библиотека устанавливается одной из команд
```bash
pip install pycontracts
pip3 install pycontracts
conda insttall pycontracts
```
[Ссылка](http://andreacensi.github.io/contracts/index.html#introduction) на домашнюю страницу библиотеки.

Предусловия и постусловия удобно оформлять не через утверждения, а как контракт функции. Нам поможет декоратор `contract` из библиотеки `PyContracts`:

### Проверка предусловий
Пример контракта с проверкой предусловий:

In [ ]:
from contracts import contract

@contract(x='int,>=0')
def f(x):
    pass

В этом случае вызов функции с недопустимым значением аргумента приведёт к исключению ContractNotRespected, а трассировка стека будет сопровождена следующей полезной информацией:

In [ ]:
f(-2)

In [ ]:
f(0.5)

### Проверка постусловий

Пример контракта с проверкой результата работы функции:

In [ ]:
@contract(returns='int,>=0')
def f(x):
    return x

In [ ]:
f(-1)

### Три варианта описания контракта функции

1. Через декоратор `contract`, как показано выше.

2. Через описание в документ строке. Можно перечислить условия в документ-строке, записанной в RST-style. Про стили документ-строки можно почитать [здесь](https://realpython.com/documenting-python-code/).

In [ ]:
import numpy
from contracts import contract

@contract
def matrix_multiply(a,  b):
    ''' Multiplies two matrices together.

        :param a: The first matrix. Must be a 2D array.
         :type a: array[MxN],M>0,N>0

        :param b: The second matrix. Must be of compatible dimensions.
         :type b: array[NxP],P>0

          :rtype: array[MxP]
    '''
    return numpy.dot(a, b)

3. Через аннотацию типов.<br>

In [ ]:
@contract
def f3(n: 'int,>=0') -> 'int,>=0':
    pass

### Язык описания контрактов PyContracts

**Логическое И**: если нужно проверить несколько условий, их можно просто записать через запятую:

In [ ]:
@contract(x='>=0,<=1')
def f(x):
    pass

**Логическое ИЛИ**: вертикальная черта:

In [ ]:
@contract(x='<0|>1')
def f(x):
    pass

@contract(x='(int|float),>=0')
def f(x):
    pass

Для **списков** возможны требования как к длине, так и к типу элементов и их значениям:

`list[length contract](elements contract)`

**Примеры:**

`list[>0]` — непустой список.

`list(int)` — список целых чисел, возможно пустой.

`list(int,>0)` — список положительных целых, возможно пустой.

`list[>0,<=100](int,>0,<=1000)` — непустой список из не более ста положительных целых чисел, не превышающих по значению тысячу.

Для **словарей** также можно ввести требования к их размеру, а также к типу ключа и/или типу значения:

`dict[length contract](key contract: value contract)`

**Примеры:**

`dict[>0]` — непустой словарь.

`dict(str:*)` — словарь со строками в качестве ключей и любыми типами значений.

`dict[>0](str:(int,>0))` — непустой словарь с ключами-строками и положительными целочисленными значениями.

### Описание нового контракта

При помощи декоратора можно создать новый вид контракта:

In [ ]:
@new_contract
def even(x):
    if x % 2 != 0:
        msg = 'The number %d is not even.' % x
        raise ValueError(msg)

После этого его можно использовать как и обычный:

In [ ]:
@contract(x='int,even')
def foo(x):
    pass

После этого его можно использовать как и обычный:

In [ ]:
@contract(x='int,even')
def foo(x):
    pass

Можно создать новый вид контракта и так:

In [ ]:
new_contract('short_list', 'list[N],N>0,N<=10')

@contract(a='short_list')
def bubble_sort(a):
    for bypass in range(len(a)-1):
        for i in range(len(a)-1-bypass):
            if a[i] > a[i+1]:
                a[i], a[i+1] = a[i+1], a[i]

### Связывание значений различных параметров
В языке описания контрактов PyContracts используются переменные:

- строчные латинские буквы — для любых объектов

- заглавные латинские буквы — для целых чисел

Пример такой связки:

In [ ]:
@contract(words='list[N](str),N>0',
          returns='list[N](>=0)')
def get_words_lengths(words):
    return [len(word) for word in words]

В этом примере контракт проверит не только то, что возвращается тип list, но и то, что этот список имеет ту же длину, что и переданный ей список words.

### Упражнение 1. Векторное произведение

Допишите контракт к функции `cross_product()`. Добавьте в функцию внутренние инварианты.

In [ ]:
from contracts import contract


def cross_product(a, b):
    cx = a[1]*b[2] - a[2]*b[1]
    cy = a[2]*b[0] - a[0]*b[2]
    cz = a[0]*b[1] - a[1]*b[0]
    return [cx, cy, cz]

In [ ]:
import numpy as np

a = np.array([1, 0, 0])
b = [0, 1]
cross_product(a, b)

### Упражнение 2. Алгоритм Евклида

Напишите функцию для поиска наибольшего общего делителя согласно алгоритму Евклида. Используйте библиотеку `PyContracts` для проверки параметров функции и возвращаемого значения.

# <font color=blue>Исключения</font>

В своей основе исключения предназначены для аварийного завершения программы, но гибкая система работы с ними предоставляет много незаменимых приемов, которые будут рассмотрены в этом разделе.

Исключения - это специальные объекты, которые можно "возбудить" с помощью ключевого слова `raise`. Процедура возбуждение включает:

- сохранение информации об исключении;

- последовательный выход из всех блоков кода путем прерывания их выполнения.

**Информация об исключении** включает.

- traceback - последовательность функций, выполнение которых прерывается исключением. Для каждой функции указывается, на какой строчке кода она остановилась.

- Состояние стека вызовов.

- Тип исключения.

- Само исключение.

Эту информацию можно получить с помощью функции [`sys.exc_info()`](https://docs.python.org/3/library/sys.html#sys.exc_info), но на практике это практически никогда не нужно.

Пример **выхода из программы** при возбуждении исключения.

Рассмотрим  код
```python
def g():
    for _ in S:
        stmt3
        stmt4
        raise exc
        stmt5

def f():
    if cond:
        stmt1
        stmt2
        g()

f()
```
Исключение произошло в цикле `for` функции `g()`, поэтому сначала будет произведен выход из блока `for`, затем из функции `g()`. Вызов функции `g()` произошел внутри блока `if`, поэтому производится выход из блока `if` и далее из функции `f()`. Завершение программы осуществляется после выхода из блока самого высокого уровня в месте вызова `f()`. 

**Исключение может быть возбуждено в следующих случаях**

1. Интерпретатор не может выполнить какую-либо инструкцию. В таких случаях исключение возбуждается интерпретатором.

- Преждевременная остановка программы пользователем с помощью клавиатуры или мышки (исключение `KeyboardInterrupt`).

- Для реализации некоторых функций языка неявно используются специальные исключения, например: `StopIteration` для реализации итераторов, `GeneratorExit` для выхода из генератора и некоторые другие.

- Программист может прописать возбуждение исключения для случаев, когда входные данные не соответствуют требованиям или вычислительных ресурсов не хватает. Также возможно использование служебных исключений, например для реализации итераторов (`StopIteration`)

##  <font color=green>Перехватывание исключений</font>

С помощью специальной синтаксической конструкции
```python
try:
    statements1
except ErrorType1 as e:
    statements2
except ErrorType2 as e:
    statements3
else:
    statements4
finally:
    statements5
```

Если исключение произошло внутри блока `try`, то перед выходом из блока проверяется тип исключения. Если тип возбужденного исключения 

- совпадет с типом, указанным после одного из ключевых слов `except`

- или окажется подклассом указанного в `except` типа, 

то выход из программы отменяется. Вместо этого выполняется код в соответствующем блоке `except`.

1. Типы исключений проверяются в том порядке, в котором указаны, и выполняется только первый из `except` блоков с подходящим типом исключения.

2. Блок `else` выполняется только, если исключение не было возбуждено в блоке `try`.

3. Блок `finally` будет выполнен в любом случае, независимо от того, было исключение во от типа возбужденного исключения

In [40]:
class My(TypeError):
    pass

l = []
try:
    raise ValueError
except My:
    print('my')
except TypeError:
    print('except')
else:
    print('else')
finally:
    print('finally')

finally


ValueError: 